In [2]:
import nltk
#nltk.download()
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
import re
from nltk.stem import WordNetLemmatizer
data = pd.read_csv('Dataset.csv')
label_encoder = preprocessing.LabelEncoder()
data ['accuracy'] = label_encoder.fit_transform(data['accuracy'])
student_answer = data['studentAnswer']
ref_answers = data['referenceAnswer']
ref_1= data['ref_1']
ps = PorterStemmer()
#answers_arr = answers.to_numpy()
#ex_sentence = answers[0]
stop_words = set(stopwords.words("english"))

In [3]:
def clean(s):
  out = word_tokenize(s)
  ans = ' '.join([''.join(ps.stem(word)) for word in out if word not in stop_words])
  return ans
filtered_answers = [clean(s) for s in student_answer]

In [4]:
print(filtered_answers[0])
print(student_answer[0])

By let sit dish day .
By letting it sit in a dish for a day.


In [5]:
def clean(s):
  out = word_tokenize(s)
  ans = ' '.join([''.join(ps.stem(word)) for word in out if word not in stop_words])
  return ans
filtered_answers_ref = [clean(s) for s in ref_1]

In [6]:
print(filtered_answers_ref[0])
print(ref_1[0])

We evapor salt water water evapor separ .
We evaporated the salt from the water because the water evaporated and separated.


In [7]:
answers= filtered_answers+filtered_answers_ref

In [8]:
l=len(filtered_answers)

In [9]:
test_sent = answers
# print(test_sent)
cv = TfidfVectorizer()
tfidf = cv.fit_transform(test_sent)
students_answers = tfidf[:l]
model_answers = tfidf[l:]

In [10]:
from sklearn.metrics.pairwise import cosine_similarity
for x in range(20):
    print("Cosine Similarity comparing student answer" , x, "and model answer",x , "is" ,cosine_similarity(students_answers[x], model_answers[x]))

Cosine Similarity comparing student answer 0 and model answer 0 is [[0.]]
Cosine Similarity comparing student answer 1 and model answer 1 is [[0.47921638]]
Cosine Similarity comparing student answer 2 and model answer 2 is [[0.57421108]]
Cosine Similarity comparing student answer 3 and model answer 3 is [[0.08860209]]
Cosine Similarity comparing student answer 4 and model answer 4 is [[0.10502184]]
Cosine Similarity comparing student answer 5 and model answer 5 is [[0.50214509]]
Cosine Similarity comparing student answer 6 and model answer 6 is [[0.]]
Cosine Similarity comparing student answer 7 and model answer 7 is [[0.63303424]]
Cosine Similarity comparing student answer 8 and model answer 8 is [[0.64692536]]
Cosine Similarity comparing student answer 9 and model answer 9 is [[0.48199351]]
Cosine Similarity comparing student answer 10 and model answer 10 is [[0.36787392]]
Cosine Similarity comparing student answer 11 and model answer 11 is [[0.57027885]]
Cosine Similarity comparing 

In [11]:
#Just to check same value or not
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(students_answers[3], model_answers[3])

array([[0.08860209]])

In [12]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(students_answers[4], model_answers[4])

array([[0.10502184]])

In [13]:
threshold = 0.01
predictions = []
for i in range(l):
  predictions.append(*(cosine_similarity(students_answers[i], model_answers[i]) > threshold)[0])

In [14]:
predictions = [1 if predictions[i] == True else 0 for i in range(len(predictions))]

In [17]:
x_train, x_test, y_train, y_test = train_test_split(data['studentAnswer'], data['accuracy'], random_state=0,test_size = 0.25)

In [18]:
print(data['accuracy'].value_counts())

1    2944
0    1981
Name: accuracy, dtype: int64


In [19]:
print("x_train shape: {}".format(x_train.shape), end='\n')
print("y_train shape: {}".format(y_train.shape), end='\n\n')
print("x_test shape: {}".format(x_test.shape), end='\n')
print("y_test shape: {}".format(y_test.shape), end='\n\n')

x_train shape: (3693,)
y_train shape: (3693,)

x_test shape: (1232,)
y_test shape: (1232,)



In [20]:
x_train

2864    I think she made a solution because the eggshe...
685     Overly Orange and Luscious Lemon have more vit...
2260    Maybe Greta has a bigger hand and Andre smalle...
3935    The blue covered got hotter and in the same am...
221                  The salt crystals look like squares.
                              ...                        
4859    They need to have thrown it at the same place ...
3264                           It will make a doom sound.
1653    C because it is on the square part and on the ...
2607    The alum has the same pattern and it looks the...
2732    Because they are both at the highest point of ...
Name: studentAnswer, Length: 3693, dtype: object

In [22]:
# Fitting and transforming the training data to a document-term matrix using TfidfVectorizer 
tfidf = TfidfVectorizer(min_df=5) #minimum document frequency of 5
X_train_tfidf = tfidf.fit_transform(x_train)
print("Number of features : %d \n" %len(tfidf.get_feature_names())) #1722
print("Show some feature names : \n", tfidf.get_feature_names()[::1000])

Number of features : 824 

Show some feature names : 
 ['10']


In [24]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
mnb.fit(X_train_tfidf, y_train)

MultinomialNB()

In [28]:
X_test = tfidf.fit_transform(x_test)

In [30]:
y_pred =mnb.predict(X_test)

ValueError: dimension mismatch

In [14]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

0.4619289340101523


In [15]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

0.6101794645483966 0.704483695652174 0.5381421899325376


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
ConfMat=confusion_matrix(y_test, y_pred)
print(ConfMat)

In [ ]:
import seaborn as sns
ax = sns.heatmap(ConfMat/np.sum(ConfMat),fmt='.2%', annot=True, cmap='Blues')

ax.set_title(' Confusion Matrix \n\n');
ax.set_xlabel('\n Predicted')
ax.set_ylabel('Actual ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['1', '2'])
ax.yaxis.set_ticklabels(['1', '2',])

## Display the visualization of the Confusion Matrix.
##plt.show()